In [36]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import string
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem import WordNetLemmatizer

import numpy as np
import re
import nltk
import spacy
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
# Load the CSV file
df = pd.read_csv('/content/Tag_dataset_tags.csv')

# Data Cleaning
df.dropna(subset=['Description'], inplace=True)  # Drop rows with missing descriptions
df.drop_duplicates(subset=['Description'], keep='first', inplace=True)  # Remove duplicates

# Text Preprocessing with Lemmatization and Stopword Removal
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(lemmatized_tokens)

df['lemmatized_description'] = df['Description'].apply(lemmatize_text)

# Remove Stopwords from Lemmatized Text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    return ' '.join(filtered_tokens)

df['cleaned_lemmatized_description'] = df['lemmatized_description'].apply(remove_stopwords)

# Create Array of Words from Cleaned Lemmatized Description
def create_word_array(text):
    return word_tokenize(text)

df['word_array'] = df['cleaned_lemmatized_description'].apply(create_word_array)

# Perform POS Tagging on Each Word in Each Array
def pos_tag_words(word_array):
    return pos_tag(word_array)

df['pos_tagged_words'] = df['word_array'].apply(pos_tag_words)

# Create Separate Columns for Specific POS Tags
def extract_pos_tags(tagged_words, pos_tag):
    return ' '.join([word for word, tag in tagged_words if tag == pos_tag])

df['Nouns'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'NN') + ' ' + extract_pos_tags(x, 'NNS') + ' ' + extract_pos_tags(x, 'NNP') + ' ' + extract_pos_tags(x, 'NNPS'))
df['Verbs'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'VB') + ' ' + extract_pos_tags(x, 'VBD') + ' ' + extract_pos_tags(x, 'VBG') + ' ' + extract_pos_tags(x, 'VBN') + ' ' + extract_pos_tags(x, 'VBP') + ' ' + extract_pos_tags(x, 'VBZ'))
df['Adjectives'] = df['pos_tagged_words'].apply(lambda x: extract_pos_tags(x, 'JJ'))
# Save the preprocessed data with specific POS tagged words to a new CSV file
df.to_csv('Final_pd.csv', index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [37]:
import tensorflow as tf
from tensorflow import keras

In [38]:
df.head()

,ID,Name,Description,Tags,lemmatized_description,cleaned_lemmatized_description,word_array,pos_tagged_words,Nouns,Verbs,Adjectives
0,1.0,Sarah,Sarah is a dedicated student with a passion fo...,Physics,Sarah is a dedicated student with a passion fo...,Sarah dedicated student passion Mathematics Ph...,"[Sarah, dedicated, student, passion, Mathemati...","[(Sarah, NNP), (dedicated, VBD), (student, NN)...",student passion interest lie realm fervor curi...,dedicated mystery,universe
1,2.0,Alex,Alex is a tech enthusiast studying Computer Sc...,Computer Science,Alex is a tech enthusiast studying Computer Sc...,Alex tech enthusiast studying Computer Science...,"[Alex, tech, enthusiast, studying, Computer, S...","[(Alex, NNP), (tech, VBD), (enthusiast, RB), (...",interest revolve technology computing Alex Co...,tech studying shaping explores,cutting-edge future
2,3.0,Emily,"Emily is an avid reader and historian, focusin...",Literature,"Emily is an avid reader and historian , focusi...","Emily avid reader historian , focusing Literat...","[Emily, avid, reader, historian, ,, focusing, ...","[(Emily, RB), (avid, JJ), (reader, NN), (histo...",reader historian inspiration endeavor Literat...,find focusing drawing,avid joy deep narrative creative
3,4.0,James,James is a science enthusiast excelling in Bio...,Chemistry,James is a science enthusiast excelling in Bio...,James science enthusiast excelling Biology Che...,"[James, science, enthusiast, excelling, Biolog...","[(James, NNP), (science, NN), (enthusiast, NN)...",science enthusiast interest research experimen...,excelling drive seek,unravel molecular
4,5.0,Mia,Mia is a student of Economics and Political Sc...,Economics,Mia is a student of Economics and Political Sc...,"Mia student Economics Political Science , pass...","[Mia, student, Economics, Political, Science, ...","[(Mia, NNP), (student, NN), (Economics, NNP), ...",student passionate understanding interest comm...,making informed reflect,global dynamic positive


In [60]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Read the CSV file into a pandas DataFrame
file_path = "/content/Final_pd.csv"
data = pd.read_csv(file_path)

# Function to extract tags from description
def extract_tags(description):
    # Tokenize the description
    tokens = word_tokenize(description)
    # Perform Part-of-Speech tagging
    tagged_words = pos_tag(tokens)
    # Define your logic to extract tags, for simplicity, let's assume the first noun encountered is the tag
    tags = [word for word, tag in tagged_words if tag.startswith('N')]
    return tags

# Apply the function to each row to generate tags
data['Tags'] = data['Description'].apply(extract_tags)

# Save the updated DataFrame to a new CSV file if needed
output_file_path = "/content/Final_pd_with_tags.csv"
data.to_csv(output_file_path, index=False)

# Display the updated DataFrame
print(data)


      ID       Name                                        Description  \
0    1.0      Sarah  Sarah is a dedicated student with a passion fo...   
1    2.0       Alex  Alex is a tech enthusiast studying Computer Sc...   
2    3.0      Emily  Emily is an avid reader and historian, focusin...   
3    4.0      James  James is a science enthusiast excelling in Bio...   
4    5.0        Mia  Mia is a student of Economics and Political Sc...   
5    6.0       Ryan  Ryan is an aspiring engineer fascinated by Rob...   
6    7.0     Sophia  Sophia is a psychology enthusiast exploring th...   
7    8.0      Lucas  Lucas is an art aficionado studying Art Histor...   
8    9.0     Ethan   Ethan is a linguistics and anthropology studen...   
9   10.0     Olivia  Olivia is a budding environmental scientist sp...   
10  11.0       Lily  Lily is a dedicated student focusing on Chemis...   
11  12.0        Max  Max is an aspiring Computer Engineering studen...   
12  13.0        Ava  Ava delves into t

In [61]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Read the CSV file into a pandas DataFrame
file_path = "/content/Final_pd.csv"
data = pd.read_csv(file_path)

# Function to extract tags from description
def extract_tags(description):
    # Tokenize the description
    tokens = word_tokenize(description)
    # Perform Part-of-Speech tagging
    tagged_words = pos_tag(tokens)
    # Define your logic to extract tags, for simplicity, let's assume the first noun encountered is the tag
    tags = [word for word, tag in tagged_words if tag.startswith('N')]
    return tags

# Create a dictionary to map names to tags
name_tag_dict = {}

# Iterate over each row to generate tags and map them to names
for index, row in data.iterrows():
    name = row['Name']
    description = row['Description']
    tags = extract_tags(description)
    name_tag_dict[name] = tags

# Display the dictionary
print(name_tag_dict)


{'Sarah': ['Sarah', 'student', 'passion', 'Mathematics', 'Physics', 'interests', 'realms', 'Astrophysics', 'Quantum', 'Mechanics', 'mysteries', 'universe', 'fervor', 'curiosity'], 'Alex': ['Alex', 'enthusiast', 'Computer', 'Science', 'Data', 'Science', 'interests', 'Artificial', 'Intelligence', 'Machine', 'Learning', 'cutting-edge', 'technologies', 'future'], 'Emily': ['reader', 'Literature', 'History', 'joy', 'Creative', 'Writing', 'narratives', 'Ancient', 'Civilizations', 'inspiration', 'past', 'endeavors'], 'James': ['James', 'science', 'enthusiast', 'excelling', 'Biology', 'Chemistry', 'interests', 'Genetics', 'Biochemistry', 'research', 'experiments', 'complexities', 'life', 'level'], 'Mia': ['Mia', 'student', 'Economics', 'Political', 'Science', 'passionate', 'dynamics', 'interests', 'International', 'Relations', 'Public', 'Policy', 'commitment', 'impact', 'society', 'decision-making'], 'Ryan': ['Ryan', 'engineer', 'Robotics', 'Renewable', 'Energy', 'interests', 'Systems', 'techn

In [64]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from sklearn.pipeline import Pipeline

# Read the CSV file into a pandas DataFrame
file_path = "/content/Final_pd.csv"
data = pd.read_csv(file_path)

# Function to extract tags from description
def extract_tags(description):
    # Tokenize the description
    tokens = word_tokenize(description)
    # Perform Part-of-Speech tagging
    tagged_words = pos_tag(tokens)
    # Define your logic to extract tags, for simplicity, let's assume the first noun encountered is the tag
    tags = [word for word, tag in tagged_words if tag.startswith('N')]
    return ' '.join(tags)

# Preprocess data
data['tags'] = data['Description'].apply(extract_tags)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['Description'], data['tags'], test_size=0.2, random_state=42)

# Create a pipeline with CountVectorizer and Multinomial Naive Bayes
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('clf', MultinomialNB())])

# Train the model
text_clf.fit(X_train, y_train)

# Test the model
predicted = text_clf.predict(X_test)

# Print accuracy
print("Accuracy:", text_clf.score(X_test, y_test))

# Example usage: Predict tags for a given phrase
def predict_tags(phrase):
    predicted_tags = text_clf.predict([phrase])
    return predicted_tags

# Example usage
phrase = "Student is very much interested in Mathematics and Physics"
predicted_tags = predict_tags(phrase)
print("Predicted tags for the phrase:", predicted_tags)


Accuracy: 0.0
Predicted tags for the phrase: ['Sarah student passion Mathematics Physics interests realms Astrophysics Quantum Mechanics mysteries universe fervor curiosity']


NEW APPROACH

In [36]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import string
nltk.download('stopwords')
nltk.download('wordnet')
# Assuming 'Description' is a column in your DataFrame
# Load your dataset
df = pd.read_csv("/content/Final_pd.csv")

# Initialize NLTK tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_and_tokenize(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    words = word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return tokens

# Apply preprocessing
df['Processed'] = df['cleaned_lemmatized_description'].apply(preprocess_and_tokenize)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
def score_terms(tokens):
    # Count term frequencies
    term_freq = Counter(tokens)
    # Sort terms by frequency
    sorted_terms = dict(sorted(term_freq.items(), key=lambda item: item[1], reverse=True))
    return sorted_terms

# Score and rank terms in descriptions
df['Ranked_Terms'] = df['Processed'].apply(score_terms)


In [38]:
def select_top_n_tags(ranked_terms, n=7):
    # Select the top N terms
    return list(ranked_terms.keys())[:n]

df['Top_7_Tags'] = df['Ranked_Terms'].apply(lambda x: select_top_n_tags(x, 7))
df.head()


,ID,Name,Description,Tags,lemmatized_description,cleaned_lemmatized_description,word_array,pos_tagged_words,Nouns,Verbs,Adjectives,Processed,Ranked_Terms,Top_7_Tags
0,1.0,Sarah,Sarah is a dedicated student with a passion fo...,Physics,Sarah is a dedicated student with a passion fo...,Sarah dedicated student passion Mathematics Ph...,"['Sarah', 'dedicated', 'student', 'passion', '...","[('Sarah', 'NNP'), ('dedicated', 'VBD'), ('stu...",student passion interest lie realm fervor curi...,dedicated mystery,universe,"[sarah, dedicated, student, passion, mathemati...","{'sarah': 1, 'dedicated': 1, 'student': 1, 'pa...","[sarah, dedicated, student, passion, mathemati..."
1,2.0,Alex,Alex is a tech enthusiast studying Computer Sc...,Computer Science,Alex is a tech enthusiast studying Computer Sc...,Alex tech enthusiast studying Computer Science...,"['Alex', 'tech', 'enthusiast', 'studying', 'Co...","[('Alex', 'NNP'), ('tech', 'VBD'), ('enthusias...",interest revolve technology computing Alex Co...,tech studying shaping explores,cutting-edge future,"[alex, tech, enthusiast, studying, computer, s...","{'science': 2, 'alex': 1, 'tech': 1, 'enthusia...","[science, alex, tech, enthusiast, studying, co..."
2,3.0,Emily,"Emily is an avid reader and historian, focusin...",Literature,"Emily is an avid reader and historian , focusi...","Emily avid reader historian , focusing Literat...","['Emily', 'avid', 'reader', 'historian', ',', ...","[('Emily', 'RB'), ('avid', 'JJ'), ('reader', '...",reader historian inspiration endeavor Literat...,find focusing drawing,avid joy deep narrative creative,"[emily, avid, reader, historian, focusing, lit...","{'creative': 2, 'emily': 1, 'avid': 1, 'reader...","[creative, emily, avid, reader, historian, foc..."
3,4.0,James,James is a science enthusiast excelling in Bio...,Chemistry,James is a science enthusiast excelling in Bio...,James science enthusiast excelling Biology Che...,"['James', 'science', 'enthusiast', 'excelling'...","[('James', 'NNP'), ('science', 'NN'), ('enthus...",science enthusiast interest research experimen...,excelling drive seek,unravel molecular,"[james, science, enthusiast, excelling, biolog...","{'james': 1, 'science': 1, 'enthusiast': 1, 'e...","[james, science, enthusiast, excelling, biolog..."
4,5.0,Mia,Mia is a student of Economics and Political Sc...,Economics,Mia is a student of Economics and Political Sc...,"Mia student Economics Political Science , pass...","['Mia', 'student', 'Economics', 'Political', '...","[('Mia', 'NNP'), ('student', 'NN'), ('Economic...",student passionate understanding interest comm...,making informed reflect,global dynamic positive,"[mia, student, economics, political, science, ...","{'mia': 1, 'student': 1, 'economics': 1, 'poli...","[mia, student, economics, political, science, ..."


In [39]:
# Map names to their top 5 tags
name_to_top_tags = pd.Series(df['Top_7_Tags'].values, index=df['Name']).to_dict()

print(name_to_top_tags)


{'Sarah': ['sarah', 'dedicated', 'student', 'passion', 'mathematics', 'physic', 'interest'], 'Alex': ['science', 'alex', 'tech', 'enthusiast', 'studying', 'computer', 'data'], 'Emily': ['creative', 'emily', 'avid', 'reader', 'historian', 'focusing', 'literature'], 'James': ['james', 'science', 'enthusiast', 'excelling', 'biology', 'chemistry', 'interest'], 'Mia': ['mia', 'student', 'economics', 'political', 'science', 'passionate', 'understanding'], 'Ryan': ['ryan', 'aspiring', 'engineer', 'fascinated', 'robotics', 'renewable', 'energy'], 'Sophia': ['sophia', 'dedicated', 'student', 'focusing', 'sociology', 'gender', 'study'], 'Lucas': ['art', 'lucas', 'enthusiast', 'studying', 'fine', 'sculpture', 'interest'], 'Ethan ': ['genetic', 'engineering', 'biotechnology', 'ethan', 'academic', 'journey', 'biology'], 'Olivia': ['environmental', 'olivia', 'budding', 'scientist', 'specializing', 'science', 'geology'], 'Lily': ['chemistry', 'environmental', 'lily', 'dedicated', 'student', 'focusing

In [31]:
from collections import defaultdict

# This defaultdict will store counts and names for each tag
tag_to_overall_count_and_names = defaultdict(lambda: {"count": 0, "names": defaultdict(int)})

for name, tags in name_to_top_tags.items():
    for tag in tags:
        # Increase the overall count for the tag
        tag_to_overall_count_and_names[tag]["count"] += 1
        # Increase the count for this tag under this specific name
        tag_to_overall_count_and_names[tag]["names"][name] += 1

# Now, printing results as per your format:
for tag, info in tag_to_overall_count_and_names.items():
    names_counts = ', '.join([f"{name} - {count}" for name, count in info["names"].items()])
    print(f"{tag.capitalize()} - {names_counts}; Total - {info['count']}")


Sarah - Sarah - 1; Total - 1
Dedicated - Sarah - 1, Sophia - 1, Lily - 1; Total - 3
Student - Sarah - 1, Mia - 1, Sophia - 1, Lily - 1, Max - 1; Total - 5
Passion - Sarah - 1, Grace - 1, Charlotte - 1; Total - 3
Mathematics - Sarah - 1; Total - 1
Physic - Sarah - 1, Leo - 1; Total - 2
Interest - Sarah - 1, James - 1, Lucas - 1; Total - 3
Science - Alex - 1, James - 1, Mia - 1, Olivia - 1, Lily - 1, Zoe - 1, Isabella - 1, Gia - 1; Total - 8
Alex - Alex - 1; Total - 1
Tech - Alex - 1, Oliver - 1; Total - 2
Enthusiast - Alex - 1, James - 1, Lucas - 1, Oliver - 1, William - 1; Total - 5
Studying - Alex - 1, Lucas - 1, Oliver - 1; Total - 3
Computer - Alex - 1, Max - 1, Oliver - 1; Total - 3
Data - Alex - 1; Total - 1
Creative - Emily - 1, Charlotte - 1; Total - 2
Emily - Emily - 1; Total - 1
Avid - Emily - 1; Total - 1
Reader - Emily - 1; Total - 1
Historian - Emily - 1; Total - 1
Focusing - Emily - 1, Sophia - 1, Lily - 1; Total - 3
Literature - Emily - 1, Charlotte - 1; Total - 2
James -

In [42]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from collections import defaultdict

# Assuming data is loaded into a DataFrame `data`
data = pd.read_csv("/content/Final_pd.csv")

# Custom stopwords
custom_stopwords = {'dedicated', 'life', 'passion', 'interest'}  # Extend as needed
stop_words = set(stopwords.words('english')).union(custom_stopwords)

# Function to tokenize, remove custom stopwords and verbs
def tokenize_and_filter(text):
    tokens = word_tokenize(text.lower())  # Tokenize and lower case
    filtered_tokens = [word for word in tokens if word not in stop_words]  # Remove custom stopwords
    # POS tagging and filter out verbs (keep nouns, NN; you might also keep adjectives, JJ)
    tagged = pos_tag(filtered_tokens)
    nouns_adjectives = [word for word, tag in tagged if tag.startswith('NN') or tag.startswith('JJ')]
    return ' '.join(nouns_adjectives)

# Apply the function to the descriptions
data['filtered_description'] = data['Description'].apply(tokenize_and_filter)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit TF-IDF on the filtered descriptions
tfidf_matrix = tfidf_vectorizer.fit_transform(data['filtered_description'])

# Extract feature names to map back to words
features = tfidf_vectorizer.get_feature_names_out()

### Step 3: Extracting Top 7 Tags

def get_top_n_tags(row_data, features, n=7):
    # Get indices sorted by value in descending order
    sorted_indices = row_data.argsort()[-n:][::-1]
    return [features[i] for i in sorted_indices]

# Extract top 7 tags for each row
data['Top_7_Tags'] = [get_top_n_tags(row, features) for row in tfidf_matrix.toarray()]

### Step 4: Map Names to Tags and Count

name_to_top_tags = pd.Series(data['Top_7_Tags'].values, index=data['Name']).to_dict()

print(name_to_top_tags)

tag_to_names = defaultdict(lambda: defaultdict(int))

# Populate tag_to_names
for name, tags in name_to_top_tags.items():
    for tag in tags:
        tag_to_names[tag][name] += 1

# Consolidate counts and print
for tag, names in tag_to_names.items():
    count = sum(names.values())  # Total count for the tag
    names_str = ', '.join([f"{name} {names[name]}" for name in names])
    print(f"{tag.capitalize()} - {names_str}; Total: {count}")


{'Sarah': ['astrophysics', 'quantum', 'mechanics', 'curiosity', 'fervor', 'sarah', 'mathematics'], 'Alex': ['science', 'machine', 'learning', 'data', 'computing', 'intelligence', 'alex'], 'Emily': ['creative', 'deep', 'historian', 'avid', 'joy', 'narratives', 'reader'], 'James': ['experiments', 'molecular', 'unravel', 'james', 'level', 'chemistry', 'genetics'], 'Mia': ['policy', 'mia', 'positive', 'commitment', 'impact', 'society', 'passionate'], 'Ryan': ['projects', 'engineer', 'autonomous', 'friendly', 'renewable', 'eco', 'energy'], 'Sophia': ['structures', 'gender', 'intersectionality', 'equality', 'social', 'societal', 'sociology'], 'Lucas': ['art', 'explores', 'traditional', 'installation', 'sculpture', 'boundaries', 'inspire'], 'Ethan ': ['genetic', 'biotechnology', 'engineering', 'applications', 'manipulation', 'advancements', 'realm'], 'Olivia': ['environmental', 'scientist', 'natural', 'earth', 'propel', 'generations', 'geology'], 'Lily': ['chemistry', 'environmental', 'knowle